In [1]:
import timeit
import pandas as pd
import os
from scipy.io import savemat
t0 = timeit.default_timer()


In [ ]:
#Specify file name
file = r'C:\Users\atrip\Research\Model-Interpretation-main\Data\HN_P2C_20241212.csv'
checkpoints = 1
df = pd.read_csv(file)
t1 = timeit.default_timer()
print("Opened file in: "+ str(t1 - t0) + " seconds.")
print(df.columns)
print(df['WLI_4'].unique())

C:\Users\atrip\AppData\Local\Temp\ipykernel_35860\756110595.py:4: DtypeWarning: Columns (16,17,19,20,24,25,26,27,29,31,32,33,34) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file)


Opened file in: 36.0507465 seconds.
Index(['SubjectID', 'IRB', 'Instrument', 'PrimaryAnatomy',
       'Patient_LevelInclude_Exclude', 'Setup', 'Age', 'Sex', 'Ethnicity',
       'Race',
       ...
       'Phasor_SH3_ch2_1', 'Phasor_GH1_ch3_1', 'Phasor_SH1_ch3_1',
       'Phasor_GH2_ch3_1', 'Phasor_SH2_ch3_1', 'Phasor_GH3_ch3_1',
       'Phasor_SH3_ch3_1', 'Tob_label', 'Alc_label', 'ML_label'],
      dtype='object', length=125)
[ 0 20 31 25  2 42 19 11 17 38 24 28 37 35  9 34  8 10 15 12 22 27 36 29
 30  5 40 46 41]


In [29]:
# #Using a .mat file

# import h5py

# with h5py.File('e:\AYUSH\Large File Downloads\HN_P2C_20240522_no_iRFcap_no_CFDBug.mat', 'r') as file:
#     group_key = list(file.keys())[2]
#     data = list(file[group_key])

#     # Convert to DataFrame
#     df = pd.DataFrame(data)

# print("hi")
# print(df.columns)

In [30]:
#shows first few dataplots in df
#print(df.info())

In [3]:
#removes In Vivo filtering

filtered_data = df[df['ScanContext'] == 'In Vivo']

t1 = timeit.default_timer()
print("Checkpoint #" + str(checkpoints) +  " at: "+str(t1-t0)+ " seconds.")
checkpoints += 1 


Checkpoint #1 at: 92.80776239999999 seconds.


In [4]:
#Relabelling for sanity:

df.rename(columns={'TobbacoStatus': 'TobaccoStatus'}, inplace=True)

if('snr_ch1_1' in df.columns):
    df.rename(columns={'WLI_4':'Label','snr_ch1_1':'snr_ch1', 'snr_ch2_1':'snr_ch2', 'snr_ch3_1':'snr_ch3', 'gain_ch1_1':'gain_ch1',  'gain_ch2_1':'gain_ch2',  'gain_ch3_1':'gain_ch3' , 'spec_int_ch1_1':'spec_int_ch1', 'spec_int_ch2_1':'spec_int_ch2', 'spec_int_ch3_1':'spec_int_ch3'},inplace = True)
    df.rename(columns={'lifet_avg_ch1_1':'lifet_avg_ch1','lifet_avg_ch2_1':'lifet_avg_ch2', 'lifet_avg_ch3_1':'lifet_avg_ch3', 'int_ratio_ch1_1':'int_ratio_ch1', 'int_ratio_ch2_1':'int_ratio_ch2', 'int_ratio_ch3_1':'int_ratio_ch3'},inplace=True)

print(df.columns)

Index(['SubjectID', 'IRB', 'Instrument', 'PrimaryAnatomy',
       'Patient_LevelInclude_Exclude', 'Setup', 'Age', 'Sex', 'Ethnicity',
       'Race',
       ...
       'Phasor_SH3_ch2_1', 'Phasor_GH1_ch3_1', 'Phasor_SH1_ch3_1',
       'Phasor_GH2_ch3_1', 'Phasor_SH2_ch3_1', 'Phasor_GH3_ch3_1',
       'Phasor_SH3_ch3_1', 'Tob_label', 'Alc_label', 'ML_label'],
      dtype='object', length=125)


In [5]:
#drops columns 13-28
df.drop(df.columns[12:28], axis=1)

t1 = timeit.default_timer()
print("Checkpoint #" + str(checkpoints) + " at: "+str(t1-t0)+ " seconds.")
checkpoints += 1 

Checkpoint #2 at: 99.92249869999999 seconds.


In [34]:
# #Remove certain patients
# df = df[df['Case'] != 124]
# df = df[df['Case'] < 160]

In [6]:
#removes CH4 data
df = df[df['DataChannelsUsed'] != 'CH4']

In [7]:

#SNR Thresholding (previous was  > 30/50 respectively, now do > 40/40)
SNR_Thr_V4 = 40
SNR_Thr_FB = 40


In [8]:
#removes V4/FLImBrush data

mask = (
    ((df['snr_ch1'] < SNR_Thr_V4) & (df['Instrument'] == 'V4')) |
    ((df['snr_ch2'] < SNR_Thr_V4) & (df['Instrument'] == 'V4')) |
    ((df['snr_ch3'] < SNR_Thr_V4) & (df['Instrument'] == 'V4'))
)

df = df[~mask]

mask2 = (
    ((df['snr_ch1'] < SNR_Thr_FB) & (df['Instrument'] == 'FLImBrush')) |
    ((df['snr_ch2'] < SNR_Thr_FB) & (df['Instrument'] == 'FLImBrush')) |
    ((df['snr_ch3'] < SNR_Thr_FB) & (df['Instrument'] == 'FLImBrush'))
)

df = df[~mask2]

t1 = timeit.default_timer()
print("Checkpoint #" + str(checkpoints) + " at: "+str(t1-t0)+ " seconds.")
checkpoints += 1 

Checkpoint #3 at: 129.8944514 seconds.


In [9]:
# Remove rows with missing entries in specific columns (NaN values)
df = df.dropna(subset=['lifet_avg_ch1', 'lifet_avg_ch2', 'lifet_avg_ch3'])
df = df.dropna(subset=['int_ratio_ch1', 'int_ratio_ch2', 'int_ratio_ch3'])

#remove data with the 0 label
df = df[df['Label'] != 0]

#Remove data with negative int values
df = df[~((df['lifet_avg_ch1'] < 0) | (df['lifet_avg_ch2'] < 0) | (df['lifet_avg_ch3'] < 0))]
df = df[~((df['spec_int_ch1'] < 0) | (df['spec_int_ch2'] < 0) | (df['spec_int_ch3'] < 0))]

t1 = timeit.default_timer()
print("Checkpoint #" + str(checkpoints) + " at: "+str(t1-t0)+ " seconds.")
checkpoints += 1 

Checkpoint #4 at: 134.9205241 seconds.


In [10]:
# Normalizing intensity ratios for V4 to be comparable to FLImBrush

total_spec_int = df['spec_int_ch1'] + df['spec_int_ch2'] + df['spec_int_ch3']
IR_ch1 = 0.1877 * (df['spec_int_ch1'] / total_spec_int)
IR_ch2 = 0.8108 * (df['spec_int_ch2'] / total_spec_int)
IR_ch3 = 1.4116 * (df['spec_int_ch3'] / total_spec_int)

## Calculate Int_ratio_ch1, Int_ratio_ch2, and Int_ratio_ch3
total_IR = IR_ch1 + IR_ch2 + IR_ch3
Int_ratio_ch1 = IR_ch1 / total_IR
Int_ratio_ch2 = IR_ch2 / total_IR
Int_ratio_ch3 = IR_ch3 / total_IR

condition = df['Instrument'] == 'V4'
df.loc[condition, 'int_ratio_ch1'] = Int_ratio_ch1[condition]
df.loc[condition, 'int_ratio_ch2'] = Int_ratio_ch2[condition]
df.loc[condition, 'int_ratio_ch3'] = Int_ratio_ch3[condition]

t1 = timeit.default_timer()
print("Checkpoint #" + str(checkpoints) + "at: "+str(t1-t0)+ " seconds.")
checkpoints += 1 

Checkpoint #5at: 143.99553070000002 seconds.


In [11]:
#Additional Filtering for gain

df = df[df['gain_ch1'] < 300]
df = df[df['gain_ch2'] < 500]
df = df[df['gain_ch3'] < 500]

In [12]:
print("Saving to file...")
# Gets dir and saves to local folder 
script_dir = './HN_P2C_files/'
file_base = 'modified_' + os.path.basename(file)

if not os.path.exists(script_dir):
    # Create the directory
    os.makedirs(script_dir)
    print(f"Directory '{script_dir}' was created.")

output_file_path = os.path.join(script_dir + file_base)
# df.to_csv(output_file_path, index=False)
# #deletes old csv(comment out if not needed)
# file_to_delete = os.path.join(script_dir, file)
# if os.path.exists(file_to_delete):

#for saving as csv 
df.to_csv(output_file_path, index=False)

#for saving as mat(takes longer but smaller file size)
#file_base = 'modified_' + os.path.basename(file)[:-4] + '.mat'
# data_to_save = {col: df[col].values for col in df.columns}
# savemat(output_file_path, data_to_save)

t1 = timeit.default_timer()
print("Saved to " + output_file_path + ". Finished in " + str(t1-t0) + " seconds.")

Saving to file...
Directory './HN_P2C_files/' was created.
Saved to ./HN_P2C_files/modified_ml_labelled_HN_P2C_20241212.csv. Finished in 245.0700636 seconds.
